In [1]:
from pathlib import Path
import pandas as pd

from rekall.predicates import _area, _iou, overlaps
from stsearch.stdlib import tiou
from virat_helper import load_events, LIST_CLIP_ID, EVENTTYPE

In [2]:
print(LIST_CLIP_ID)

['VIRAT_S_010000_08_000893_001024', 'VIRAT_S_010000_03_000442_000528', 'VIRAT_S_010201_02_000167_000197', 'VIRAT_S_010003_01_000111_000137', 'VIRAT_S_050000_11_001530_001576', 'VIRAT_S_010202_06_000784_000873', 'VIRAT_S_000201_01_000384_000589', 'VIRAT_S_010113_07_000965_001013', 'VIRAT_S_050000_06_000908_000970', 'VIRAT_S_000201_06_001354_001397', 'VIRAT_S_010203_02_000347_000397', 'VIRAT_S_010201_08_000705_000739', 'VIRAT_S_010205_02_000301_000345', 'VIRAT_S_010203_00_000047_000139', 'VIRAT_S_010109_04_000487_000538', 'VIRAT_S_010107_02_000282_000312', 'VIRAT_S_040001_02_001102_001530', 'VIRAT_S_050203_06_001202_001264', 'VIRAT_S_040103_07_001011_001093', 'VIRAT_S_010003_09_000779_000861', 'VIRAT_S_010001_04_000583_000646', 'VIRAT_S_050000_08_001235_001295', 'VIRAT_S_040003_01_000083_000193', 'VIRAT_S_010111_09_000981_001014', 'VIRAT_S_010113_06_000831_000896', 'VIRAT_S_050300_08_001705_001771', 'VIRAT_S_050300_05_001272_001330', 'VIRAT_S_010200_06_000702_000744', 'VIRAT_S_040000_05_

In [3]:
def parse_event_list(clip_id, event_type) -> pd.DataFrame:
    try:
        df = load_events(clip_id)
    except FileNotFoundError:
        # some clips have no events and no event file
        return pd.DataFrame()
    
    df = df[df['event_type']==event_type]
    df['x2'] = df['x1'] + df['w']
    df['y2'] = df['y1'] + df['h']
    df['t1'], df['t2'] = df['start_frame'], df['end_frame']
    
    rv = []
    for event_id in df['event_id'].unique():
        edf = df[df['event_id']==event_id]
        rv.append({
            'event_id': event_id,
            'x1': edf['x1'].min(),
            'x2': edf['x2'].max(),
            'y1': edf['y1'].min(),
            'y2': edf['y2'].max(),
            't1': edf['t1'].min(),
            't2': edf['t2'].max(),
        })
        
    return  pd.DataFrame(rv)

def parse_all_event_list(event_type) -> pd.DataFrame:
    all_df = []
    for clip_id in LIST_CLIP_ID:
        df = parse_event_list(clip_id, event_type)
        if not df.empty:
            df['clip_id'] = clip_id
            all_df.append(df)
        
    return pd.concat(all_df, ignore_index=True)
    

# gt = parse_event_list('VIRAT_S_000202_00_000000_000977', EVENTTYPE.GET_OUT_VEHICLE)
GT = parse_all_event_list(EVENTTYPE.GET_OUT_VEHICLE)
print(GT)

    event_id   x1    x2   y1   y2    t1    t2                          clip_id
0         10  383   546  146  331  1135  1271  VIRAT_S_050000_11_001530_001576
1          0  695   767  362  420   631  1022  VIRAT_S_000201_01_000384_000589
2          0  132   213  352  435   449   701  VIRAT_S_000201_06_001354_001397
3          0  686   786  353  434   572   758  VIRAT_S_000201_03_000640_000672
4          2  423   609  509  690  3340  3517                   VIRAT_S_000006
..       ...  ...   ...  ...  ...   ...   ...                              ...
92         0  786  1034  501  711  2486  2619                   VIRAT_S_000002
93         0  529   643   58  117   656   745  VIRAT_S_050203_01_000147_000223
94         1  454   583   84  154  1067  1125  VIRAT_S_050203_01_000147_000223
95         0  660   771  475  515   637   832  VIRAT_S_000200_00_000100_000171
96         0  685   767  336  379  1180  1296  VIRAT_S_000203_07_001341_001458

[97 rows x 8 columns]


In [4]:
def parse_result(result_file="getoutcar.csv"):
    df = pd.read_csv(result_file, index_col=0)
    # convert relative coord to pixel coord
    df['x1'] = df['x1'] * df['width']
    df['x2'] = df['x2'] * df['width']
    df['y1'] = df['y1'] * df['height']
    df['y2'] = df['y2'] * df['height']
    return df

prediction = parse_result()
print(prediction)

                              clip_id     t1     t2           x1           x2  \
0                      VIRAT_S_000002   2342   2642   713.980808  1135.593418   
1                      VIRAT_S_000002   3362   3662   713.980808  1094.813690   
2                      VIRAT_S_000002   4022   4322   713.980808  1358.815935   
3                      VIRAT_S_000102  23432  23732   376.719904   917.400520   
4                      VIRAT_S_000102  23972  24272   376.719904   906.766220   
...                               ...    ...    ...          ...          ...   
1335  VIRAT_S_050301_02_000544_000607    272    572  1453.315063  1550.991898   
1336  VIRAT_S_050301_02_000544_000607    542    842   693.333374   862.021751   
1337  VIRAT_S_050301_02_000544_000607    692    992   694.725895   862.021751   
1338  VIRAT_S_050301_02_000544_000607    812   1112  1450.549049  1543.575668   
1339  VIRAT_S_050301_02_000544_000607   1232   1532   694.725895   862.021751   

              y1          y

In [20]:
def is_hit(i1, i2):
#     return _iou(i1, i2) > 0.01 and overlaps()(i1, i2)
    return _iou(i1, i2) > 0.01 and tiou(i1, i2)>0.1

In [21]:
# Evaluate precision and recall
GT['hit'] = False
prediction['hit'] = False

for i, pred in prediction.iterrows():
    for j, gt in GT[GT['clip_id']==pred['clip_id']].iterrows():
        if is_hit(pred, gt):
            prediction.loc[i, ['hit']] = True
            GT.loc[j, ['hit']] = True
            
print(GT)


    event_id   x1    x2   y1   y2    t1    t2  \
0         10  383   546  146  331  1135  1271   
1          0  695   767  362  420   631  1022   
2          0  132   213  352  435   449   701   
3          0  686   786  353  434   572   758   
4          2  423   609  509  690  3340  3517   
..       ...  ...   ...  ...  ...   ...   ...   
92         0  786  1034  501  711  2486  2619   
93         0  529   643   58  117   656   745   
94         1  454   583   84  154  1067  1125   
95         0  660   771  475  515   637   832   
96         0  685   767  336  379  1180  1296   

                            clip_id    hit  
0   VIRAT_S_050000_11_001530_001576   True  
1   VIRAT_S_000201_01_000384_000589   True  
2   VIRAT_S_000201_06_001354_001397   True  
3   VIRAT_S_000201_03_000640_000672   True  
4                    VIRAT_S_000006   True  
..                              ...    ...  
92                   VIRAT_S_000002   True  
93  VIRAT_S_050203_01_000147_000223  False  
94  VI

In [22]:
GT_hit = len(GT[GT['hit']==True])
prediction_hit = len(prediction[prediction['hit']==True])
print(f"Precision={prediction_hit}/{len(prediction)}={prediction_hit/len(prediction):.2f}")
print(f"Recall={GT_hit}/{len(GT)}={GT_hit/len(GT):.2f}")
print(f"Video length={sum(prediction['t2']-prediction['t1'])/30/3600} hours")

Precision=106/1340=0.08
Recall=60/97=0.62
Video length=3.445851851851852 hours


In [8]:
# w/o bidirectional tracking, we find 812 results. w/ bidirectional tracking we find 812 results.

In [9]:
# FPs
print((prediction[prediction['hit']==False][['clip_id', 't1','t2', 'result_size']]).head(50))

                            clip_id     t1     t2  result_size
1                    VIRAT_S_000002   3362   3662      1168510
2                    VIRAT_S_000002   4022   4322      3192702
4                    VIRAT_S_000102  23972  24272      1926622
7                    VIRAT_S_000003  10622  10922      1134324
8                    VIRAT_S_000003  12092  12392      1197215
9                    VIRAT_S_000003  12332  12632      1264435
10                   VIRAT_S_000003  12632  12932      1758829
12                   VIRAT_S_000001  18272  18572       490592
15                   VIRAT_S_000004   7352   7652      1133157
18                   VIRAT_S_000101   2522   2822      1314010
20                   VIRAT_S_000101  16952  17252      1113370
21                   VIRAT_S_000006    392    692      2074305
24                   VIRAT_S_000006   3842   4142      1573921
25                   VIRAT_S_000006   4562   4862      1292437
26                   VIRAT_S_000006   5132   5432      

In [10]:
prediction[prediction['clip_id']=='VIRAT_S_050300_05_001272_001330']

,clip_id,t1,t2,x1,x2,y1,y2,result_size,frame_count,fps,width,height,hit
1329,VIRAT_S_050300_05_001272_001330,0,92,703.745155,856.506557,444.532210,508.041598,88490,1737,30,1920,1080,False
1330,VIRAT_S_050300_05_001272_001330,242,542,1424.884644,1537.947693,204.225555,320.485793,328325,1737,30,1920,1080,False
1331,VIRAT_S_050300_05_001272_001330,482,782,703.745155,856.506557,444.532210,513.243042,308835,1737,30,1920,1080,False
1332,VIRAT_S_050300_05_001272_001330,662,962,703.745155,856.506557,442.393158,508.041598,297988,1737,30,1920,1080,False
1333,VIRAT_S_050300_05_001272_001330,992,1292,703.745155,856.506557,443.537537,512.996063,309784,1737,30,1920,1080,False
1334,VIRAT_S_050300_05_001272_001330,1142,1442,703.745155,856.506557,444.532210,513.574577,316722,1737,30,1920,1080,False
